<a href="https://colab.research.google.com/github/skywalker0803r/works/blob/master/bert_dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip uninstall googletrans
#!pip install googletrans==3.1.0a0

In [2]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Amazing Gift for Nick1 (1) (1).xlsx')
df

,speak_id,Content,r1score,r2score,r3score,r4score,response1,response2,response3,response4,keywords,intention,emotionlabel,f4,event,entity,relation
0,Student test1,你覺得尼克一開始的反應怎麼樣,1.0,NaN,NaN,NaN,i think nick 's initial reaction was a little ...,What does Nick start to do?,disgust,Who starts to play with the baby?,NaN,SetQ,neutral,Feedback,NaN,NaN,NaN
1,Student test2,"真是過分,怎麼可以欺負小寶寶",0.0,NaN,NaN,NaN,i think it is too much . it is a little bit to...,What do Nick's mom and dad say to the baby?,"Nick smiles and says, I'm a big brother",What kind of robot does the baby crawl to?,NaN,Statement,fear,Nothing,NaN,NaN,NaN
2,Student test1,可能是因為他沒辦法接受吧,0.0,NaN,NaN,NaN,but he does not understand . it is a very smal...,"Who shouts: ""Don't touch it. That's my robot!""",That's not my brother,What does Nick's mom say to the baby?,NaN,Statement,sadness,Feeling,NaN,NaN,NaN
3,Student test2,因為母愛要被分走了?,-1.0,NaN,NaN,NaN,"it is too late , he is only a baby .",What kind of robot does the baby crawl to?,"Nick smiles and says, I'm a big brother",Nick looks back at Nick with what?,NaN,PropQ,surprise,Nothing,NaN,NaN,NaN
4,Student test1,如果是我的話 突然有一個新的人加進我的家庭 我也會覺得很奇怪,0.0,-1.0,NaN,NaN,that is understandable . i hope it is not for ...,What does Nick say to the baby?,I'm a big brother,What does Nick look at the baby with?,NaN,Statement,surprise,Feeling,NaN,NaN,NaN
5,Student test2,太可憐了,NaN,NaN,1.0,NaN,oh no ! that must be so scary .,What does the baby look back at Nick with?,It is so nice to take care of others,What does Nick say the baby points to a toy gi...,NaN,Statement,sadness,Nothing,NaN,NaN,NaN
6,Student test1,"而且還要承擔責任,好慘,你看尼克最一開始還想要把他丟了",1.0,NaN,NaN,NaN,i understand . i am sure he would have been de...,What does Nick want to do to the baby?,It is so nice to take care of others,What does Nick say to the baby?,NaN,Statement,disgust,Feeling,NaN,NaN,NaN
7,Student test2,但是丟到垃圾桶真的太過分了,1.0,NaN,NaN,NaN,that is too bad . i would have been furious,What does Nick say the baby points to a toy gi...,Nick wants to put the baby in a trash can and ...,What does Nick start to do?,NaN,Statement,disgust,Future,NaN,NaN,NaN
8,Student test1,還想了各種方法,-1.0,NaN,NaN,-1.0,"that is good , but there is a lot of work to b...",What do Nick's mom and dad say when they come ...,It is so nice to take care of others,What does Nick say to the baby?,NaN,Statement,neutral,Feeling,NaN,NaN,NaN
9,Student test2,太壞了,-1.0,NaN,-1.0,NaN,that would be a good idea . what would you do ?,What do the baby crawls to Nick and opens his ...,That's not my brother,What does the baby look back at Nick with?,NaN,Statement,disgust,Future,NaN,NaN,NaN


In [3]:
df.columns

Index(['speak_id', 'Content', 'r1score', 'r2score', 'r3score', 'r4score',
       'response1', 'response2', 'response3', 'response4', 'keywords',
       'intention', 'emotionlabel', 'f4', 'event', 'entity', 'relation'],
      dtype='object')

In [4]:
df[['Content', 'r1score', 'r2score', 'r3score', 'r4score',
       'response1', 'response2', 'response3', 'response4',
       'intention', 'emotionlabel', 'f4']]

,Content,r1score,r2score,r3score,r4score,response1,response2,response3,response4,intention,emotionlabel,f4
0,你覺得尼克一開始的反應怎麼樣,1.0,NaN,NaN,NaN,i think nick 's initial reaction was a little ...,What does Nick start to do?,disgust,Who starts to play with the baby?,SetQ,neutral,Feedback
1,"真是過分,怎麼可以欺負小寶寶",0.0,NaN,NaN,NaN,i think it is too much . it is a little bit to...,What do Nick's mom and dad say to the baby?,"Nick smiles and says, I'm a big brother",What kind of robot does the baby crawl to?,Statement,fear,Nothing
2,可能是因為他沒辦法接受吧,0.0,NaN,NaN,NaN,but he does not understand . it is a very smal...,"Who shouts: ""Don't touch it. That's my robot!""",That's not my brother,What does Nick's mom say to the baby?,Statement,sadness,Feeling
3,因為母愛要被分走了?,-1.0,NaN,NaN,NaN,"it is too late , he is only a baby .",What kind of robot does the baby crawl to?,"Nick smiles and says, I'm a big brother",Nick looks back at Nick with what?,PropQ,surprise,Nothing
4,如果是我的話 突然有一個新的人加進我的家庭 我也會覺得很奇怪,0.0,-1.0,NaN,NaN,that is understandable . i hope it is not for ...,What does Nick say to the baby?,I'm a big brother,What does Nick look at the baby with?,Statement,surprise,Feeling
5,太可憐了,NaN,NaN,1.0,NaN,oh no ! that must be so scary .,What does the baby look back at Nick with?,It is so nice to take care of others,What does Nick say the baby points to a toy gi...,Statement,sadness,Nothing
6,"而且還要承擔責任,好慘,你看尼克最一開始還想要把他丟了",1.0,NaN,NaN,NaN,i understand . i am sure he would have been de...,What does Nick want to do to the baby?,It is so nice to take care of others,What does Nick say to the baby?,Statement,disgust,Feeling
7,但是丟到垃圾桶真的太過分了,1.0,NaN,NaN,NaN,that is too bad . i would have been furious,What does Nick say the baby points to a toy gi...,Nick wants to put the baby in a trash can and ...,What does Nick start to do?,Statement,disgust,Future
8,還想了各種方法,-1.0,NaN,NaN,-1.0,"that is good , but there is a lot of work to b...",What do Nick's mom and dad say when they come ...,It is so nice to take care of others,What does Nick say to the baby?,Statement,neutral,Feeling
9,太壞了,-1.0,NaN,-1.0,NaN,that would be a good idea . what would you do ?,What do the baby crawls to Nick and opens his ...,That's not my brother,What does the baby look back at Nick with?,Statement,disgust,Future


In [5]:
import numpy as np
label = df[['r1score', 'r2score', 'r3score', 'r4score']].fillna(-999)
label = pd.get_dummies(label.T.apply(np.argmax)).values
label

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]], dtype=uint8)

In [6]:
df[['Content','intention', 'emotionlabel', 'f4']]

,Content,intention,emotionlabel,f4
0,你覺得尼克一開始的反應怎麼樣,SetQ,neutral,Feedback
1,"真是過分,怎麼可以欺負小寶寶",Statement,fear,Nothing
2,可能是因為他沒辦法接受吧,Statement,sadness,Feeling
3,因為母愛要被分走了?,PropQ,surprise,Nothing
4,如果是我的話 突然有一個新的人加進我的家庭 我也會覺得很奇怪,Statement,surprise,Feeling
5,太可憐了,Statement,sadness,Nothing
6,"而且還要承擔責任,好慘,你看尼克最一開始還想要把他丟了",Statement,disgust,Feeling
7,但是丟到垃圾桶真的太過分了,Statement,disgust,Future
8,還想了各種方法,Statement,neutral,Feeling
9,太壞了,Statement,disgust,Future


In [7]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

inputs = ["a", "a"*20, "a"*100, "abcede"*200]
for i in inputs:
  inputs = tokenizer(i, padding='max_length', truncation=True, max_length = 15, return_tensors="pt")
  print(inputs["input_ids"])
  outputs = model(**inputs)
  print(outputs.last_hidden_state.shape, i, len(i))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[ 101, 1037,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0]])
torch.Size([1, 15, 768]) a 1
tensor([[  101, 13360, 11057, 11057, 11057, 11057, 11057, 11057, 11057, 11057,
          2050,   102,     0,     0,     0]])
torch.Size([1, 15, 768]) aaaaaaaaaaaaaaaaaaaa 20
tensor([[  101, 13360, 11057, 11057, 11057, 11057, 11057, 11057, 11057, 11057,
         11057, 11057, 11057, 11057,   102]])
torch.Size([1, 15, 768]) aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa 100
tensor([[101, 100, 102,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0]])
torch.Size([1, 15, 768]) abcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcedeabcede

In [8]:
import googletrans
translator = googletrans.Translator()
results = translator.translate('我覺得今天天氣不好')
print(results)
print(results.text)

Translated(src=zh-CN, dest=en, text=I think the weather is bad today, pronunciation=None, extra_data="{'translat...")
I think the weather is bad today


In [25]:
class preprocess_model(object):
  def __init__(self):
    super().__init__()
    self.translator = googletrans.Translator()
    self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    self.bert_model = AutoModel.from_pretrained("bert-base-uncased")

  def __call__(self,sentence):
    x = self.translator.translate(sentence).text
    print(x)
    x = self.tokenizer(x, padding='max_length', truncation=True, max_length = 15, return_tensors="pt")
    return self.bert_model(**x).last_hidden_state

In [32]:
預處理模型 = preprocess_model()
預處理模型('弟弟不要,brother no')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


brother no, brother no


tensor([[[-0.2653,  0.2819,  0.0119,  ..., -0.2877,  0.3570,  0.4862],
         [ 0.2039,  0.3870,  0.2616,  ...,  0.0293,  0.5112,  0.2558],
         [ 0.8031,  0.2979,  0.6042,  ..., -0.0736, -0.1005,  0.4912],
         ...,
         [-0.0808,  0.2199,  0.7160,  ...,  0.1606,  0.0412,  0.2598],
         [ 0.0307,  0.1381,  0.7306,  ...,  0.2688,  0.1162,  0.1458],
         [-0.0668,  0.1335,  0.7694,  ...,  0.1918,  0.0175,  0.2275]]],
       grad_fn=<NativeLayerNormBackward0>)

In [42]:
x = ''.join(df[['Content','intention', 'emotionlabel', 'f4']].head(1).values[0])
print(x)
display(預處理模型(x))
print(預處理模型(x).shape)

你覺得尼克一開始的反應怎麼樣SetQneutralFeedback
What do you think of Nick's initial reaction to SetQneutralFeedback


tensor([[[ 0.1086, -0.0541, -0.3922,  ..., -0.2677,  0.3984,  0.4692],
         [ 0.8846, -0.7155, -1.0731,  ...,  0.3281,  0.1995, -0.7106],
         [ 0.3012, -0.2888, -0.9050,  ..., -0.5869,  0.2942, -0.5813],
         ...,
         [ 0.8034, -0.6023,  0.3285,  ..., -0.2790, -0.5152, -0.1156],
         [-0.2881, -0.6625, -0.5675,  ...,  0.1832,  0.5746, -0.2088],
         [ 0.7851, -0.1216, -0.2028,  ...,  0.4046, -0.7411, -0.1535]]],
       grad_fn=<NativeLayerNormBackward0>)

What do you think of Nick's initial reaction to SetQneutralFeedback
torch.Size([1, 15, 768])
